In [1]:
import numpy as np
from lattice import Lattice2D, BrickwallLattice, SquareLattice

In [2]:
l = Lattice2D(
    dimensions=(6, 12),
    t_hop=1,
    E_amplitude=lambda t: np.cos(t),
    E_dir=np.array([1, 0]),
    h=1e-3,
    T=12,
    initial_occupation=0.5,
)

l.evolve(options={"progress_bar": True})
l.save_current_density_animation("polarized-lattice.gif", sample_every=97, curl_norm=l.maximum_curl, fps=15)

Creating Hamiltonians...
Done.
Calculating energy eigenstates...
Done.
50 % of states were set as initially occupied.
10.0%. Run time:   8.29s. Est. time left: 00:00:01:14
20.0%. Run time:  12.46s. Est. time left: 00:00:00:49
30.0%. Run time:  19.40s. Est. time left: 00:00:00:45
40.0%. Run time:  24.74s. Est. time left: 00:00:00:37
50.0%. Run time:  29.99s. Est. time left: 00:00:00:29
60.0%. Run time:  37.20s. Est. time left: 00:00:00:24
70.0%. Run time:  41.39s. Est. time left: 00:00:00:17
80.0%. Run time:  47.95s. Est. time left: 00:00:00:11
90.0%. Run time:  54.17s. Est. time left: 00:00:00:06
100.0%. Run time:  58.21s. Est. time left: 00:00:00:00
Total run time:  58.21s


Generating animation:  99%|█████████▉| 122/123 [00:48<00:00,  2.52frame/s]


In [3]:
l = BrickwallLattice(
    dimensions=(7, 12),
    t_hop=1,
    E_amplitude=lambda t: np.cos(t),
    E_dir=np.array([1, 0]),
    h=2e-3,
    T=12,
    initial_occupation=0.5,
)

l.evolve(options={"progress_bar": True})
l.save_current_density_animation("polarized-bw-lattice.gif", sample_every=97, curl_norm=l.maximum_curl, fps=15)

Creating Hamiltonians...
Done.
Calculating energy eigenstates...
Done.
50 % of states were set as initially occupied.
10.0%. Run time:  15.29s. Est. time left: 00:00:02:17
20.0%. Run time:  23.38s. Est. time left: 00:00:01:33
30.0%. Run time:  35.52s. Est. time left: 00:00:01:22
40.0%. Run time:  44.99s. Est. time left: 00:00:01:07
50.0%. Run time:  55.74s. Est. time left: 00:00:00:55
60.0%. Run time:  70.70s. Est. time left: 00:00:00:47
70.0%. Run time:  80.51s. Est. time left: 00:00:00:34
80.0%. Run time:  95.28s. Est. time left: 00:00:00:23
90.0%. Run time: 108.94s. Est. time left: 00:00:00:12
100.0%. Run time: 119.54s. Est. time left: 00:00:00:00
Total run time: 119.54s


Generating animation:  98%|█████████▊| 60/61 [00:23<00:00,  2.56frame/s]


In [26]:
class BrickwallLattice(Lattice2D):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.cell_path = np.array([(0, 1), (1, 2), (2, self.Lx+2), (self.Lx+2, self.Lx+1), (self.Lx+1, self.Lx), (self.Lx, 0)])


    def get_brickwall_lattice_sites(self) -> np.ndarray:
        if self.Lx % 2 == 0:
            y_hop_row = np.tile([0, 1], self.Lx//2)
            y_hop = np.concatenate([y_hop_row, 1 - y_hop_row] * (self.Ly // 2 - 1) + [y_hop_row])
        else:
            y_hop = np.tile([0, 1], (self.Ly-1)*self.Lx//2)
            y_hop = np.concatenate([y_hop, [0] * (1- self.Ly % 2)])
        
        erase_positions = np.diag(y_hop, self.Lx) + np.diag(y_hop, -self.Lx)
        return 1-erase_positions
    
    
    def create_hopping_hamiltonian(self):
        return super().create_hopping_hamiltonian() * self.get_brickwall_lattice_sites()
    
    def curl(self, J: np.ndarray) -> np.ndarray:
        cell_width = (self.cell_path.flatten() % self.Lx).max()
        cell_height = (self.cell_path.flatten() // self.Lx).max()
        curl = dict()
        # curl_row_length = self.L-cell_width

        for i in range(0, self.Ly - cell_height, cell_height):
            for j in range(i%2, self.Lx - cell_width, cell_width):
                site_index = self.Lx * i + j
                curl[site_index] = sum([J[site_index + di, site_index + dj] for di, dj in self.cell_path])
        return curl
    
    
        
    

class SquareLattice(Lattice2D):
    def __init__(self, L: int, *args, **kwargs):
        super().__init__(dimensions=(L, L), *args, **kwargs)
        self.L = L

